In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
!pip install Levenshtein  
!pip install --upgrade git+https://github.com/ariaghora/mpstemmer.git 
!pip install tensorflow_addons
!pip install transformers

!gdown --id 1y6NlbLXV41DjLioNQXYRCQTiZWSo6rXa # sinonim.json
!gdown --id 1HfYW0fHOcqYXkYoectft_BfQyrLmdBv- # dummy dataset

## import import 
import pandas as pd
import numpy as np
import tensorflow as tf
import json
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import sklearn
import gensim
import nltk
import re
import string
import tensorflow_addons as tfa
import matplotlib.cm as cm
from google.colab import drive
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense,Dropout,Embedding,Flatten,Input,LSTM,Bidirectional
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

import re

from tensorflow.keras import backend as K
from nltk.probability import FreqDist

from transformers import AutoTokenizer, TFAutoModel

import matplotlib as mpl
mpl.rcParams["figure.figsize"] = (10,8)
import numpy as np
nltk.download("punkt")
nltk.download("stopwords")

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,StratifiedKFold,RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix,classification_report

from mpstemmer import MPStemmer 

print(f"\n\npandas {pd.__version__}")
print(f"numpy {np.__version__}")
print(f"tensorflow {tf.__version__}")
print(f"Matplotlib {matplotlib.__version__}")
print(f"seaborn {sns.__version__}")
print(f"gensim {gensim.__version__}" )
print(f"nltk {nltk.__version__}" )


AUTO = tf.data.experimental.AUTOTUNE

  Cloning https://github.com/ariaghora/mpstemmer.git to /tmp/pip-req-build-u6r_daqr
  Running command git clone -q https://github.com/ariaghora/mpstemmer.git /tmp/pip-req-build-u6r_daqr
Downloading...
From: https://drive.google.com/uc?id=1y6NlbLXV41DjLioNQXYRCQTiZWSo6rXa
To: /content/sinonim.json
4.94MB [00:00, 78.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HfYW0fHOcqYXkYoectft_BfQyrLmdBv-
To: /content/TrainigData.txt
3.34MB [00:00, 106MB/s]
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


pandas 1.1.5
numpy 1.19.5
tensorflow 2.6.0
Matplotlib 3.2.2
seaborn 0.11.2
gensim 3.6.0
nltk 3.2.5


# **Data**

In [227]:
df.head()

,raw,processed,label
0,Hari ini udh mulai ppkm yaa..😣,hari ini udah mulai ppkm ya,1
1,@e100ss mohon info apakah PGS pasar turi selam...,mohon informasi apakah pgs pasar turi selama p...,1
2,@bertanyarl Di rumah aja soalnya lagi ppkm\n\n...,di rumah saja soalnya lagi ppkm tidak tahu bak...,1
3,Pangkal penanganan pandemi #covid19 di Indones...,pangkal penanganan pandemi di indonesia yang t...,0
4,ppkm mikro anjingggggggg,ppkm mikro anjing,0


In [228]:
df['label'].value_counts()

0    4498
1    4028
2    1284
Name: label, dtype: int64

In [229]:
df.head()

,raw,processed,label
0,Hari ini udh mulai ppkm yaa..😣,hari ini udah mulai ppkm ya,1
1,@e100ss mohon info apakah PGS pasar turi selam...,mohon informasi apakah pgs pasar turi selama p...,1
2,@bertanyarl Di rumah aja soalnya lagi ppkm\n\n...,di rumah saja soalnya lagi ppkm tidak tahu bak...,1
3,Pangkal penanganan pandemi #covid19 di Indones...,pangkal penanganan pandemi di indonesia yang t...,0
4,ppkm mikro anjingggggggg,ppkm mikro anjing,0


## **model**

In [230]:
max_length = 26
max_features = 250
padding_type="post"
truncating_type="post"

In [231]:
def prepareForDeepLearning(X,Y, max_features, max_lengths,oov_token,padding_type="post",truncating_type="post" ):
  Y = Y.astype(np.int32)
  tokenizer = Tokenizer(num_words = max_features, oov_token = oov_token)
  tokenizer.fit_on_texts(X)

  x_sequences = tokenizer.texts_to_sequences(X)
  
  x_padded = pad_sequences(x_sequences,maxlen = max_lengths, padding = padding_type, truncating = truncating_type)
  Y = to_categorical(Y )
  return tokenizer,x_sequences,x_padded,Y


In [247]:
def compile(model):
  """
  model : tf.keras.Model
  """
  model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer= tf.keras.optimizers.Adam(),
                  metrics = get_metrics())

  return model
def compile_imbalanced(model):
  """
  model : tf.keras.Model
  """
  model.compile(loss = tfa.losses.SigmoidFocalCrossEntropy(),optimizer= tf.keras.optimizers.Adam(),
                  metrics = get_metrics())
  
  return model
def get_metrics():
  return   [
          tf.keras.metrics.TruePositives(name='TruePositive'),
          tf.keras.metrics.FalsePositives(name='FalsePositive'),
          tf.keras.metrics.TrueNegatives(name='TrueNegative'),
          tf.keras.metrics.FalseNegatives(name='FalseNegative'), 
          tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall'),
          tf.keras.metrics.AUC(name='auc'),
        ]

def create_lstm(X,Embedding_layer,kernel_initializer = "xavier",seed = 0,num_categorical=3):
  """
    X : Input Features , np: array
    Embedding_layer : tf.keras.layers.Embbeding
    kernel_initializer : string
    seed : int --> Seed numbers
    num_categorical : int --> Total Class
  """
  if (kernel_initializer.lower() == "xavier"):
    
    input_layer = Input(shape=(X.shape[1]),dtype= tf.float32)
    embedding = Embedding_layer(input_layer)
    lstm = LSTM(units=100, activation=tf.nn.tanh, return_sequences = True)(embedding) 
    # lstm = LSTM(units=100, activation=tf.nn.tanh)(lstm) 
    flatten = Flatten()(lstm)
    fcn = Dense(512, activation= tf.nn.relu, kernel_initializer= tf.keras.initializers.GlorotNormal(seed=seed),)(flatten)
    fcn = Dropout(0.4)(fcn)
    fcn = Dense(128, activation= tf.nn.relu, kernel_initializer= tf.keras.initializers.GlorotNormal(seed=seed),)(fcn)
    
    output_layer = Dense(num_categorical, activation=tf.nn.softmax,)(fcn)

    model = Model(inputs=[input_layer], outputs=[output_layer])

    model = compile(model)

    return model

def create_Bi_lstm(X,Embedding_layer,kernel_initializer = "xavier",seed = 0,num_categorical=3):
  """
    X : Input Features , np: array
    Embedding_layer : tf.keras.layers.Embbeding
    kernel_initializer : string
    seed : int --> Seed numbers
    num_categorical : int --> Total Class
  """
  if (kernel_initializer.lower() == "xavier"):
    
    input_layer = Input(shape=(X.shape[1]),dtype= tf.float32)
    embedding = Embedding_layer(input_layer)
    lstm = Bidirectional(LSTM(units=100, activation=tf.nn.tanh, return_sequences = True))(embedding) 
    # lstm = LSTM(units=100, activation=tf.nn.tanh)(lstm) 
    flatten = Flatten()(lstm)
    fcn = Dense(512, activation= tf.nn.relu, kernel_initializer= tf.keras.initializers.GlorotNormal(seed=seed),)(flatten)
    fcn = Dropout(0.4)(fcn)
    fcn = Dense(124, activation= tf.nn.relu, kernel_initializer= tf.keras.initializers.GlorotNormal(seed=seed),)(fcn)
    
    output_layer = Dense(num_categorical, activation=tf.nn.softmax,)(fcn)

    model = Model(inputs=[input_layer], outputs=[output_layer])

    model = compile(model)

    return model

def fast_training(X,Y, model, epochs, tokenizer):
  x_train,x_val,y_train,y_val = sklearn.model_selection.train_test_split(X, Y,test_size=0.2,random_state=42,stratify=Y)
  print(x_val.shape)
  history = model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs =epochs,batch_size=64)
  y_pred = model.predict(x_val)

  print(f"Acc {model.evaluate(x_val,y_val)}")
  
  y_pred  = np.argmax(y_pred,axis=1)
  y_val = np.argmax(y_val,axis=1)
  
  missclass = np.argwhere(np.not_equal(y_val,y_pred)).ravel()
  true_classified = np.argwhere(np.equal(y_val,y_pred)).ravel()

  missclassDf = pd.DataFrame([i for i in tokenizer.sequences_to_texts(x_val[missclass])])
  missclassDf["label"] = y_val[missclass]
  missclassDf["predict"] = y_pred[missclass]

  true_classifiedDf = pd.DataFrame([i for i in tokenizer.sequences_to_texts(x_val[true_classified])])
  true_classifiedDf["label"] = y_val[true_classified]
  true_classifiedDf["predict"] = y_pred[true_classified]

  
  return model,true_classifiedDf,missclassDf,y_val,y_pred  

In [248]:
def prepare_word_embeddingsFor_DL(raw_X, min_count,vector_size,window,sg, seed,title ):
  raw_X = [f.split() for f in raw_X]
  w2v_model = gensim.models.Word2Vec(raw_X, size = vector_size , window = window, min_count= min_count, sg = sg, seed =seed )
  w2v_model.save(f"model_Gensim {title} {vector_size}.model")
  return w2v_model 


def createWordEmbeddingsDf(w2v):
  return pd.DataFrame(w2v[w2v.wv.vocab],index=list(w2v.wv.vocab))

def create_embeddingMatrix(tokenizer,max_features,w2v):

  embedding_matrix= np.zeros(shape = (len(tokenizer.word_index) + 1, max_features) )
  for word, i in tokenizer.word_index.items():
    try:
      embedding_vector = w2v.wv.get_vector(word)
      if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    except:
      embedding_matrix[i] = np.array([0] * max_features)
  
  return embedding_matrix
  

def prepare_embedding_layers(tokenizer, max_features, embedding_matrix, trainable):

  embedding_layer = Embedding(input_dim = len(tokenizer.word_index) + 1,
                              output_dim = 250,
                              weights = [embedding_matrix],
                              trainable =trainable  )
  
  return embedding_layer



In [249]:
data = ['Non_Stem_Clean','NonSteam_NonPPKM_Min2Word_Clean','NonStopword_Min2_Clean','Steam_Clean','Steam_Min2Word_Clean','Steam_NonPPKM_Clean']

for judul in data:
  df = pd.read_csv(f'/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Data/Balance/{judul}/clean_train.csv')
  print(f'{judul}______________________________')
  tokenizer,x_seq,x_padded,Y = prepareForDeepLearning(df['processed'].values,df['label'],10000,max_length,"")
  w2v = prepare_word_embeddingsFor_DL(df["processed"].values,4,max_features,5,1,10,"yow")
  embedding_matrix = create_embeddingMatrix(tokenizer,250,w2v)
  Embedding_layer = prepare_embedding_layers(tokenizer, 256, embedding_matrix,True)

  model_Bi_lstm = create_Bi_lstm(x_padded,Embedding_layer)
  mod_Bi_lstm,true, missBiLstm, y_val_Bi_lstm,y_pred_Bi_lstm = fast_training(x_padded,Y,model_Bi_lstm,5,tokenizer)
  # mod_Bi_lstm.save(f'/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModeBilLSTM_{judul}')
  print(f'              ')
  print(f'              ')
  print(f'              ')


Non_Stem_Clean______________________________


(771, 26)
Epoch 1/5
49/49 [==============================] - 35s 68ms/step - loss: 0.9055 - TruePositive: 1310.0000 - FalsePositive: 697.0000 - TrueNegative: 5465.0000 - FalseNegative: 1771.0000 - accuracy: 0.5751 - precision: 0.6527 - recall: 0.4252 - auc: 0.7593 - val_loss: 0.7796 - val_TruePositive: 455.0000 - val_FalsePositive: 189.0000 - val_TrueNegative: 1353.0000 - val_FalseNegative: 316.0000 - val_accuracy: 0.6719 - val_precision: 0.7065 - val_recall: 0.5901 - val_auc: 0.8340
Epoch 2/5
49/49 [==============================] - 2s 33ms/step - loss: 0.5791 - TruePositive: 2204.0000 - FalsePositive: 603.0000 - TrueNegative: 5559.0000 - FalseNegative: 877.0000 - accuracy: 0.7575 - precision: 0.7852 - recall: 0.7154 - auc: 0.9090 - val_loss: 0.8436 - val_TruePositive: 498.0000 - val_FalsePositive: 210.0000 - val_TrueNegative: 1332.0000 - val_FalseNegative: 273.0000 - val_accuracy: 0.6732 - val_precision: 0.7034 - val_recall: 0.6459 - val_auc: 0.8383
Epoch 3/5
49/49 [=================

(756, 26)
Epoch 1/5
48/48 [==============================] - 8s 68ms/step - loss: 0.9727 - TruePositive: 865.0000 - FalsePositive: 545.0000 - TrueNegative: 5497.0000 - FalseNegative: 2156.0000 - accuracy: 0.5154 - precision: 0.6135 - recall: 0.2863 - auc: 0.7070 - val_loss: 0.7848 - val_TruePositive: 432.0000 - val_FalsePositive: 202.0000 - val_TrueNegative: 1310.0000 - val_FalseNegative: 324.0000 - val_accuracy: 0.6495 - val_precision: 0.6814 - val_recall: 0.5714 - val_auc: 0.8278
Epoch 2/5
48/48 [==============================] - 2s 34ms/step - loss: 0.6039 - TruePositive: 2042.0000 - FalsePositive: 635.0000 - TrueNegative: 5407.0000 - FalseNegative: 979.0000 - accuracy: 0.7286 - precision: 0.7628 - recall: 0.6759 - auc: 0.8998 - val_loss: 0.7764 - val_TruePositive: 515.0000 - val_FalsePositive: 195.0000 - val_TrueNegative: 1317.0000 - val_FalseNegative: 241.0000 - val_accuracy: 0.7063 - val_precision: 0.7254 - val_recall: 0.6812 - val_auc: 0.8632
Epoch 3/5
48/48 [===================

(771, 26)
Epoch 1/5
49/49 [==============================] - 9s 67ms/step - loss: 0.8318 - TruePositive: 1552.0000 - FalsePositive: 666.0000 - TrueNegative: 5496.0000 - FalseNegative: 1529.0000 - accuracy: 0.6290 - precision: 0.6997 - recall: 0.5037 - auc: 0.8048 - val_loss: 0.7319 - val_TruePositive: 480.0000 - val_FalsePositive: 201.0000 - val_TrueNegative: 1341.0000 - val_FalseNegative: 291.0000 - val_accuracy: 0.6732 - val_precision: 0.7048 - val_recall: 0.6226 - val_auc: 0.8574
Epoch 2/5
49/49 [==============================] - 2s 33ms/step - loss: 0.5960 - TruePositive: 2160.0000 - FalsePositive: 566.0000 - TrueNegative: 5596.0000 - FalseNegative: 921.0000 - accuracy: 0.7566 - precision: 0.7924 - recall: 0.7011 - auc: 0.9045 - val_loss: 0.6808 - val_TruePositive: 550.0000 - val_FalsePositive: 166.0000 - val_TrueNegative: 1376.0000 - val_FalseNegative: 221.0000 - val_accuracy: 0.7406 - val_precision: 0.7682 - val_recall: 0.7134 - val_auc: 0.8915
Epoch 3/5
49/49 [==================

(771, 26)
Epoch 1/5
49/49 [==============================] - 9s 67ms/step - loss: 0.9274 - TruePositive: 1115.0000 - FalsePositive: 623.0000 - TrueNegative: 5539.0000 - FalseNegative: 1966.0000 - accuracy: 0.5557 - precision: 0.6415 - recall: 0.3619 - auc: 0.7468 - val_loss: 0.7951 - val_TruePositive: 363.0000 - val_FalsePositive: 146.0000 - val_TrueNegative: 1396.0000 - val_FalseNegative: 408.0000 - val_accuracy: 0.6278 - val_precision: 0.7132 - val_recall: 0.4708 - val_auc: 0.8250
Epoch 2/5
49/49 [==============================] - 2s 34ms/step - loss: 0.6183 - TruePositive: 2073.0000 - FalsePositive: 617.0000 - TrueNegative: 5545.0000 - FalseNegative: 1008.0000 - accuracy: 0.7313 - precision: 0.7706 - recall: 0.6728 - auc: 0.8951 - val_loss: 0.6805 - val_TruePositive: 528.0000 - val_FalsePositive: 169.0000 - val_TrueNegative: 1373.0000 - val_FalseNegative: 243.0000 - val_accuracy: 0.7237 - val_precision: 0.7575 - val_recall: 0.6848 - val_auc: 0.8795
Epoch 3/5
49/49 [=================

(771, 26)
Epoch 1/5
49/49 [==============================] - 8s 67ms/step - loss: 0.8931 - TruePositive: 1309.0000 - FalsePositive: 656.0000 - TrueNegative: 5506.0000 - FalseNegative: 1772.0000 - accuracy: 0.5839 - precision: 0.6662 - recall: 0.4249 - auc: 0.7677 - val_loss: 0.8064 - val_TruePositive: 410.0000 - val_FalsePositive: 186.0000 - val_TrueNegative: 1356.0000 - val_FalseNegative: 361.0000 - val_accuracy: 0.6329 - val_precision: 0.6879 - val_recall: 0.5318 - val_auc: 0.8187
Epoch 2/5
49/49 [==============================] - 2s 34ms/step - loss: 0.5471 - TruePositive: 2277.0000 - FalsePositive: 549.0000 - TrueNegative: 5613.0000 - FalseNegative: 804.0000 - accuracy: 0.7793 - precision: 0.8057 - recall: 0.7390 - auc: 0.9190 - val_loss: 0.7568 - val_TruePositive: 467.0000 - val_FalsePositive: 189.0000 - val_TrueNegative: 1353.0000 - val_FalseNegative: 304.0000 - val_accuracy: 0.6589 - val_precision: 0.7119 - val_recall: 0.6057 - val_auc: 0.8458
Epoch 3/5
49/49 [==================

(756, 26)
Epoch 1/5
48/48 [==============================] - 8s 67ms/step - loss: 0.9399 - TruePositive: 1036.0000 - FalsePositive: 592.0000 - TrueNegative: 5450.0000 - FalseNegative: 1985.0000 - accuracy: 0.5402 - precision: 0.6364 - recall: 0.3429 - auc: 0.7323 - val_loss: 0.8357 - val_TruePositive: 325.0000 - val_FalsePositive: 120.0000 - val_TrueNegative: 1392.0000 - val_FalseNegative: 431.0000 - val_accuracy: 0.6032 - val_precision: 0.7303 - val_recall: 0.4299 - val_auc: 0.8031
Epoch 2/5
48/48 [==============================] - 2s 33ms/step - loss: 0.5808 - TruePositive: 2138.0000 - FalsePositive: 560.0000 - TrueNegative: 5482.0000 - FalseNegative: 883.0000 - accuracy: 0.7603 - precision: 0.7924 - recall: 0.7077 - auc: 0.9089 - val_loss: 0.7170 - val_TruePositive: 515.0000 - val_FalsePositive: 177.0000 - val_TrueNegative: 1335.0000 - val_FalseNegative: 241.0000 - val_accuracy: 0.7183 - val_precision: 0.7442 - val_recall: 0.6812 - val_auc: 0.8771
Epoch 3/5
48/48 [==================

In [263]:
data = ['NonStopword_Stem_Min2_Clean','Non_Stem_Clean','NonSteam_NonPPKM_Min2Word_Clean','NonStopword_Min2_Clean','Steam_Clean','Steam_Min2Word_Clean','Steam_NonPPKM_Clean']

for judul in data:
  df = pd.read_csv(f'/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Data/Unbalance/{judul}/clean_train.csv')
  print(f'{judul}______________________________')
  tokenizer,x_seq,x_padded,Y = prepareForDeepLearning(df['processed'].values,df['label'],10000,max_length,"")
  w2v = prepare_word_embeddingsFor_DL(df["processed"].values,4,max_features,5,1,10,"yow")
  embedding_matrix = create_embeddingMatrix(tokenizer,250,w2v)
  Embedding_layer = prepare_embedding_layers(tokenizer, 256, embedding_matrix,True)

  model_Bi_lstm = create_Bi_lstm(x_padded,Embedding_layer)
  mod_Bi_lstm,true, missBiLstm, y_val_Bi_lstm,y_pred_Bi_lstm = fast_training(x_padded,Y,model_Bi_lstm,5,tokenizer)
  # mod_Bi_lstm.save(f'/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModeBilLSTM_{judul}')
  print(f'              ')
  print(f'              ')
  print(f'              ')


NonStopword_Stem_Min2_Clean______________________________


(1962, 26)
Epoch 1/5
123/123 [==============================] - 10s 46ms/step - loss: 0.7097 - TruePositive: 4896.0000 - FalsePositive: 1931.0000 - TrueNegative: 13765.0000 - FalseNegative: 2952.0000 - accuracy: 0.6850 - precision: 0.7172 - recall: 0.6239 - auc: 0.8598 - val_loss: 0.6198 - val_TruePositive: 1395.0000 - val_FalsePositive: 452.0000 - val_TrueNegative: 3472.0000 - val_FalseNegative: 567.0000 - val_accuracy: 0.7380 - val_precision: 0.7553 - val_recall: 0.7110 - val_auc: 0.8942
Epoch 2/5
123/123 [==============================] - 4s 33ms/step - loss: 0.4934 - TruePositive: 6089.0000 - FalsePositive: 1348.0000 - TrueNegative: 14348.0000 - FalseNegative: 1759.0000 - accuracy: 0.8012 - precision: 0.8187 - recall: 0.7759 - auc: 0.9338 - val_loss: 0.5659 - val_TruePositive: 1455.0000 - val_FalsePositive: 384.0000 - val_TrueNegative: 3540.0000 - val_FalseNegative: 507.0000 - val_accuracy: 0.7747 - val_precision: 0.7912 - val_recall: 0.7416 - val_auc: 0.9140
Epoch 3/5
123/123 [===

(1963, 26)
Epoch 1/5
123/123 [==============================] - 11s 55ms/step - loss: 0.7137 - TruePositive: 4922.0000 - FalsePositive: 1948.0000 - TrueNegative: 13752.0000 - FalseNegative: 2928.0000 - accuracy: 0.6846 - precision: 0.7164 - recall: 0.6270 - auc: 0.8585 - val_loss: 0.6657 - val_TruePositive: 1231.0000 - val_FalsePositive: 295.0000 - val_TrueNegative: 3631.0000 - val_FalseNegative: 732.0000 - val_accuracy: 0.7499 - val_precision: 0.8067 - val_recall: 0.6271 - val_auc: 0.8934
Epoch 2/5
123/123 [==============================] - 4s 33ms/step - loss: 0.4525 - TruePositive: 6268.0000 - FalsePositive: 1222.0000 - TrueNegative: 14478.0000 - FalseNegative: 1582.0000 - accuracy: 0.8208 - precision: 0.8368 - recall: 0.7985 - auc: 0.9443 - val_loss: 0.6153 - val_TruePositive: 1465.0000 - val_FalsePositive: 420.0000 - val_TrueNegative: 3506.0000 - val_FalseNegative: 498.0000 - val_accuracy: 0.7636 - val_precision: 0.7772 - val_recall: 0.7463 - val_auc: 0.9058
Epoch 3/5
123/123 [===

(1844, 26)
Epoch 1/5
116/116 [==============================] - 10s 45ms/step - loss: 0.7273 - TruePositive: 4440.0000 - FalsePositive: 1819.0000 - TrueNegative: 12927.0000 - FalseNegative: 2933.0000 - accuracy: 0.6756 - precision: 0.7094 - recall: 0.6022 - auc: 0.8530 - val_loss: 0.6489 - val_TruePositive: 1225.0000 - val_FalsePositive: 393.0000 - val_TrueNegative: 3295.0000 - val_FalseNegative: 619.0000 - val_accuracy: 0.7218 - val_precision: 0.7571 - val_recall: 0.6643 - val_auc: 0.8846
Epoch 2/5
116/116 [==============================] - 4s 32ms/step - loss: 0.4527 - TruePositive: 5890.0000 - FalsePositive: 1156.0000 - TrueNegative: 13590.0000 - FalseNegative: 1483.0000 - accuracy: 0.8189 - precision: 0.8359 - recall: 0.7989 - auc: 0.9442 - val_loss: 0.5725 - val_TruePositive: 1336.0000 - val_FalsePositive: 387.0000 - val_TrueNegative: 3301.0000 - val_FalseNegative: 508.0000 - val_accuracy: 0.7505 - val_precision: 0.7754 - val_recall: 0.7245 - val_auc: 0.9122
Epoch 3/5
116/116 [===

(1962, 26)
Epoch 1/5
123/123 [==============================] - 12s 57ms/step - loss: 0.6977 - TruePositive: 5019.0000 - FalsePositive: 1857.0000 - TrueNegative: 13839.0000 - FalseNegative: 2829.0000 - accuracy: 0.6993 - precision: 0.7299 - recall: 0.6395 - auc: 0.8657 - val_loss: 0.5994 - val_TruePositive: 1402.0000 - val_FalsePositive: 436.0000 - val_TrueNegative: 3488.0000 - val_FalseNegative: 560.0000 - val_accuracy: 0.7457 - val_precision: 0.7628 - val_recall: 0.7146 - val_auc: 0.9023
Epoch 2/5
123/123 [==============================] - 4s 33ms/step - loss: 0.4717 - TruePositive: 6179.0000 - FalsePositive: 1312.0000 - TrueNegative: 14384.0000 - FalseNegative: 1669.0000 - accuracy: 0.8086 - precision: 0.8249 - recall: 0.7873 - auc: 0.9396 - val_loss: 0.5697 - val_TruePositive: 1433.0000 - val_FalsePositive: 401.0000 - val_TrueNegative: 3523.0000 - val_FalseNegative: 529.0000 - val_accuracy: 0.7599 - val_precision: 0.7814 - val_recall: 0.7304 - val_auc: 0.9121
Epoch 3/5
123/123 [===

(1963, 26)
Epoch 1/5
123/123 [==============================] - 10s 46ms/step - loss: 0.7112 - TruePositive: 4908.0000 - FalsePositive: 1871.0000 - TrueNegative: 13829.0000 - FalseNegative: 2942.0000 - accuracy: 0.6902 - precision: 0.7240 - recall: 0.6252 - auc: 0.8606 - val_loss: 0.6191 - val_TruePositive: 1395.0000 - val_FalsePositive: 424.0000 - val_TrueNegative: 3502.0000 - val_FalseNegative: 568.0000 - val_accuracy: 0.7407 - val_precision: 0.7669 - val_recall: 0.7106 - val_auc: 0.8958
Epoch 2/5
123/123 [==============================] - 4s 33ms/step - loss: 0.4660 - TruePositive: 6215.0000 - FalsePositive: 1263.0000 - TrueNegative: 14437.0000 - FalseNegative: 1635.0000 - accuracy: 0.8144 - precision: 0.8311 - recall: 0.7917 - auc: 0.9409 - val_loss: 0.5983 - val_TruePositive: 1451.0000 - val_FalsePositive: 422.0000 - val_TrueNegative: 3504.0000 - val_FalseNegative: 512.0000 - val_accuracy: 0.7601 - val_precision: 0.7747 - val_recall: 0.7392 - val_auc: 0.9080
Epoch 3/5
123/123 [===

(1963, 26)
Epoch 1/5
123/123 [==============================] - 11s 55ms/step - loss: 0.7191 - TruePositive: 4884.0000 - FalsePositive: 1849.0000 - TrueNegative: 13851.0000 - FalseNegative: 2966.0000 - accuracy: 0.6868 - precision: 0.7254 - recall: 0.6222 - auc: 0.8574 - val_loss: 0.6297 - val_TruePositive: 1359.0000 - val_FalsePositive: 443.0000 - val_TrueNegative: 3483.0000 - val_FalseNegative: 604.0000 - val_accuracy: 0.7285 - val_precision: 0.7542 - val_recall: 0.6923 - val_auc: 0.8932
Epoch 2/5
123/123 [==============================] - 4s 33ms/step - loss: 0.4502 - TruePositive: 6246.0000 - FalsePositive: 1228.0000 - TrueNegative: 14472.0000 - FalseNegative: 1604.0000 - accuracy: 0.8168 - precision: 0.8357 - recall: 0.7957 - auc: 0.9448 - val_loss: 0.5845 - val_TruePositive: 1447.0000 - val_FalsePositive: 411.0000 - val_TrueNegative: 3515.0000 - val_FalseNegative: 516.0000 - val_accuracy: 0.7621 - val_precision: 0.7788 - val_recall: 0.7371 - val_auc: 0.9117
Epoch 3/5
123/123 [===

(1844, 26)
Epoch 1/5
116/116 [==============================] - 10s 46ms/step - loss: 0.7385 - TruePositive: 4425.0000 - FalsePositive: 1845.0000 - TrueNegative: 12901.0000 - FalseNegative: 2948.0000 - accuracy: 0.6718 - precision: 0.7057 - recall: 0.6002 - auc: 0.8488 - val_loss: 0.5940 - val_TruePositive: 1317.0000 - val_FalsePositive: 376.0000 - val_TrueNegative: 3312.0000 - val_FalseNegative: 527.0000 - val_accuracy: 0.7565 - val_precision: 0.7779 - val_recall: 0.7142 - val_auc: 0.9045
Epoch 2/5
116/116 [==============================] - 4s 33ms/step - loss: 0.4509 - TruePositive: 5901.0000 - FalsePositive: 1152.0000 - TrueNegative: 13594.0000 - FalseNegative: 1472.0000 - accuracy: 0.8223 - precision: 0.8367 - recall: 0.8004 - auc: 0.9448 - val_loss: 0.5938 - val_TruePositive: 1386.0000 - val_FalsePositive: 395.0000 - val_TrueNegative: 3293.0000 - val_FalseNegative: 458.0000 - val_accuracy: 0.7636 - val_precision: 0.7782 - val_recall: 0.7516 - val_auc: 0.9137
Epoch 3/5
116/116 [===

In [250]:
[]

[]

In [235]:
model_lstm = create_lstm(x_padded,Embedding_layer)
mod_lstm,true, missLstm, y_val,y_pred = fast_training(x_padded,Y,model_lstm,5,tokenizer)

(1962, 26)
Epoch 1/5
123/123 [==============================] - 7s 31ms/step - loss: 0.0625 - TruePositive: 6034.0000 - FalsePositive: 354.0000 - TrueNegative: 15342.0000 - FalseNegative: 1814.0000 - accuracy: 0.8842 - precision: 0.9446 - recall: 0.7689 - auc: 0.9744 - val_loss: 0.2134 - val_TruePositive: 1361.0000 - val_FalsePositive: 362.0000 - val_TrueNegative: 3562.0000 - val_FalseNegative: 601.0000 - val_accuracy: 0.7594 - val_precision: 0.7899 - val_recall: 0.6937 - val_auc: 0.8948
Epoch 2/5
123/123 [==============================] - 3s 22ms/step - loss: 0.0268 - TruePositive: 7238.0000 - FalsePositive: 185.0000 - TrueNegative: 15511.0000 - FalseNegative: 610.0000 - accuracy: 0.9559 - precision: 0.9751 - recall: 0.9223 - auc: 0.9954 - val_loss: 0.2473 - val_TruePositive: 1314.0000 - val_FalsePositive: 389.0000 - val_TrueNegative: 3535.0000 - val_FalseNegative: 648.0000 - val_accuracy: 0.7319 - val_precision: 0.7716 - val_recall: 0.6697 - val_auc: 0.8784
Epoch 3/5
123/123 [=======

In [259]:
data = ['NonStopword_Stem_Min2_Clean','Non_Stem_Clean','NonSteam_NonPPKM_Min2Word_Clean','NonStopword_Min2_Clean','Steam_Clean','Steam_Min2Word_Clean','Steam_NonPPKM_Clean']

def TF_idf(features_array,obj):
  features_array = np.array(features_array)
  mat = TF_obj.fit_transform(features_array)

  return mat.toarray(),TF_obj


### MOdelling SVC


def fastModelling(model, X, Y, title,TF_obj):
  """
  Fast modelling, TF_obj pakenya yang Global variable.

  model : Sklearn Model
  X : Input featuresa
  Y : Label
  TF_obj : TF_idf Object
  """
  x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(X,Y,test_size=0.2,random_state=42)
  model = model.fit(x_train,y_train)

  print(f"Training Acc {model.score(x_train,y_train)}")
  print("")
  y_pred = model.predict(x_test)
  print(classification_report(y_test,y_pred))


  
  return model
#svm_model, true_classified,missclassDf,y_test,y_pred = fastModelling(svm_model, Tf_data, Y, "test", TF_obj)


for judul in data:
  df = pd.read_csv(f'/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Data/Unbalance/{judul}/clean_train.csv')
  print(f'{judul}______________________________')
  
  TF_obj = sklearn.feature_extraction.text.TfidfVectorizer(max_features = 250,)
  Tf_data, TF_obj = TF_idf(df["processed"].values,TF_obj)
  model = GaussianNB()
  fastModelling(model,Tf_data,df["label"],judul,TF_obj)

  

  # mod_Bi_lstm.save(f'/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModeBilLSTM_{judul}')
  print(f'              ')
  print(f'              ')
  print(f'              ')


NonStopword_Stem_Min2_Clean______________________________
Training Acc 0.6344291539245668

              precision    recall  f1-score   support

           0       0.79      0.45      0.57       884
           1       0.64      0.76      0.69       832
           2       0.41      0.77      0.54       246

    accuracy                           0.62      1962
   macro avg       0.61      0.66      0.60      1962
weighted avg       0.67      0.62      0.62      1962

              
              
              
Non_Stem_Clean______________________________
Training Acc 0.5389808917197452

              precision    recall  f1-score   support

           0       0.79      0.39      0.52       893
           1       0.60      0.57      0.59       807
           2       0.27      0.79      0.41       263

    accuracy                           0.52      1963
   macro avg       0.56      0.58      0.50      1963
weighted avg       0.64      0.52      0.53      1963

              
         

In [262]:
data = ['NonStopword_Stem_Min2_Clean','Non_Stem_Clean','NonSteam_NonPPKM_Min2Word_Clean','NonStopword_Min2_Clean','Steam_Clean','Steam_Min2Word_Clean','Steam_NonPPKM_Clean']

for judul in data:
  df = pd.read_csv(f'/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Data/Unbalance/{judul}/clean_train.csv')
  print(f'{judul}______________________________')
  tokenizer,x_seq,x_padded,Y = prepareForDeepLearning(df['processed'].values,df['label'],10000,max_length,"")
  w2v = prepare_word_embeddingsFor_DL(df["processed"].values,4,max_features,5,1,10,"yow")
  embedding_matrix = create_embeddingMatrix(tokenizer,250,w2v)
  Embedding_layer = prepare_embedding_layers(tokenizer, 256, embedding_matrix,True)

  model_lstm = create_lstm(x_padded,Embedding_layer)
  mod_lstm,true, missLstm, y_val,y_pred = fast_training(x_padded,Y,model_lstm,5,tokenizer)
  mod_Bi_lstm.save(f'/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModelLSTM_{judul}')
  print(f'              ')
  print(f'              ')
  print(f'              ')

NonStopword_Stem_Min2_Clean______________________________


(1962, 26)
Epoch 1/5
123/123 [==============================] - 8s 36ms/step - loss: 0.6998 - TruePositive: 4914.0000 - FalsePositive: 1888.0000 - TrueNegative: 13808.0000 - FalseNegative: 2934.0000 - accuracy: 0.6899 - precision: 0.7224 - recall: 0.6261 - auc: 0.8639 - val_loss: 0.5889 - val_TruePositive: 1385.0000 - val_FalsePositive: 364.0000 - val_TrueNegative: 3560.0000 - val_FalseNegative: 577.0000 - val_accuracy: 0.7533 - val_precision: 0.7919 - val_recall: 0.7059 - val_auc: 0.9068
Epoch 2/5
123/123 [==============================] - 3s 22ms/step - loss: 0.4707 - TruePositive: 6205.0000 - FalsePositive: 1276.0000 - TrueNegative: 14420.0000 - FalseNegative: 1643.0000 - accuracy: 0.8114 - precision: 0.8294 - recall: 0.7906 - auc: 0.9396 - val_loss: 0.5848 - val_TruePositive: 1427.0000 - val_FalsePositive: 419.0000 - val_TrueNegative: 3505.0000 - val_FalseNegative: 535.0000 - val_accuracy: 0.7538 - val_precision: 0.7730 - val_recall: 0.7273 - val_auc: 0.9088
Epoch 3/5
123/123 [====

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModelLSTM_NonStopword_Stem_Min2_Clean/assets


INFO:tensorflow:Assets written to: /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModelLSTM_NonStopword_Stem_Min2_Clean/assets


              
              
              
Non_Stem_Clean______________________________


...

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModelLSTM_Steam_Min2Word_Clean/assets


              
              
              
Steam_NonPPKM_Clean______________________________


(1844, 26)
Epoch 1/5
116/116 [==============================] - 8s 34ms/step - loss: 0.7245 - TruePositive: 4449.0000 - FalsePositive: 1787.0000 - TrueNegative: 12959.0000 - FalseNegative: 2924.0000 - accuracy: 0.6742 - precision: 0.7134 - recall: 0.6034 - auc: 0.8545 - val_loss: 0.6112 - val_TruePositive: 1311.0000 - val_FalsePositive: 401.0000 - val_TrueNegative: 3287.0000 - val_FalseNegative: 533.0000 - val_accuracy: 0.7451 - val_precision: 0.7658 - val_recall: 0.7110 - val_auc: 0.8986
Epoch 2/5
116/116 [==============================] - 3s 22ms/step - loss: 0.4438 - TruePositive: 5916.0000 - FalsePositive: 1132.0000 - TrueNegative: 13614.0000 - FalseNegative: 1457.0000 - accuracy: 0.8223 - precision: 0.8394 - recall: 0.8024 - auc: 0.9463 - val_loss: 0.6344 - val_TruePositive: 1295.0000 - val_FalsePositive: 448.0000 - val_TrueNegative: 3240.0000 - val_FalseNegative: 549.0000 - val_accuracy: 0.7261 - val_precision: 0.7430 - val_recall: 0.7023 - val_auc: 0.8951
Epoch 3/5
116/116 [====

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModelLSTM_Steam_NonPPKM_Clean/assets


INFO:tensorflow:Assets written to: /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModelLSTM_Steam_NonPPKM_Clean/assets


In [236]:
mod_lstm.save('/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModelLSTM_inbalanced_Stemming_non stopword')
mod_Bi_lstm.save('/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModeBilLSTM_inbalanced_Stemming_non stopword')

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModelLSTM_inbalanced_Stemming_non stopword/assets


INFO:tensorflow:Assets written to: /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModelLSTM_inbalanced_Stemming_non stopword/assets


INFO:tensorflow:Assets written to: /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModeBilLSTM_inbalanced_Stemming_non stopword/assets


INFO:tensorflow:Assets written to: /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/ModeBilLSTM_inbalanced_Stemming_non stopword/assets


In [251]:
def get_proba_predict_deep_learning(dataFrame,model,tokenizer,minimal_kategori,maksimal_kategori,padding_type="post"):
  # menerima input dataframe yg berisi miss klasifikasi model deep learning
  # return dataframe yang memiliki proba dari predict model
  
  x_jadi = tokenizer.texts_to_sequences(dataFrame[0].values)
  
  x_jadi = pad_sequences(x_jadi,maxlen = max_length, padding = padding_type, truncating = truncating_type)
  temp = model.predict(x_jadi)

  for i in range(minimal_kategori,maksimal_kategori+1): 
    dataFrame[f'{i}'] = np.round(temp[:,i:i+1],4)
  return dataFrame


In [252]:
missProbaBiLSTM = get_proba_predict_deep_learning(missBiLstm,mod_Bi_lstm,tokenizer,0,2)
missProbaBiLSTM['predict'].value_counts()

0    92
1    68
2    42
Name: predict, dtype: int64

In [132]:
missProbaBiLSTM

,0,label,predict,0,1,2
0,ppkm sih rafathar birthday party kocak ...,0,1,0.3484,0.4367,0.2149
1,ppkm kali pengusaha berlega simak threadnya ...,2,1,0.1548,0.6780,0.1672
2,berita nya sih untuk protokol ppkm ...,1,0,0.4726,0.3510,0.1763
3,ppkm darurat kepentingan rakyat ...,0,2,0.2767,0.3816,0.3417
4,ppkm umur tahun main tetap disuruh pulang magr...,1,0,0.5596,0.2522,0.1882
...,...,...,...,...,...,...
501,solat ied ppkm iya malas sih ...,1,0,0.5404,0.3318,0.1278
502,ppkm pak salam beres ...,0,1,0.0261,0.9599,0.0140
503,sepi iya gara gara ppkm ...,1,0,0.5925,0.2339,0.1736
504,ppkm efektif nurunin kasus covid iya deh diper...,2,0,0.5221,0.2898,0.1881


In [133]:
missProbaBiLSTM.to_csv('/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Error Analysis/missBiLSTM_imbalanced_Stemming.csv',index=False)

In [134]:

def visProbaDF(missDL,minimal_kategori,maksimal_kategori):
  for i in range(minimal_kategori,maksimal_kategori+1):
    plt.figure()
    klasifikasi = i
    sns.countplot(pd.cut(missDL[f'{klasifikasi}'],bins=10))
    plt.title(f'Proba {klasifikasi}')
    plt.xlabel(f'Klasifikasi {klasifikasi}')
  plt.show()


In [ ]:
missProbaLSTM(missProbaLSTM,0,2)

In [ ]:
missProbaBiLSTM = get_proba_predict_deep_learning(missBiLstm,mod_lstm,tokenizer,0,2)
missProbaBiLSTM

In [ ]:
missProbaBiLSTM(missProbaLSTM,0,2)